# Purpose
After filtering out the spammers, we can now work on the weibo data. The goal of this purpose is to preprocess the data for indexing.

We first store all messages that contain "#" to a list since hashtags in weibo is denoted as sth like #机器学习#. 
From that list, we extract hashtag(s) from each message and save it to a list. We record the message id and its corresponding hashtag in the list called hashtags. 

In [2]:
import csv
import pandas as pd
import numpy as np
import gensim # word2vec package
import jieba  # for Chinese text segementation

C:\Anaconda2\envs\gl-env\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('normal_all2')
print len(df)
df.head()

892539


,hashtag_count,mention_count,text,time,url_count,userid
0,0,0,冷言冷语。。。继续。。。这样很好。。。,2012-09-03 00:03:19,0,uWWBCGJE4
1,0,1,傻子 过去就过去了 越想只会让自己 越来越TM看不起自己 恨自己不是吗？有什么了不起的 人不...,2012-09-03 00:04:12,0,u1O52TEBM
2,0,0,闲来无事，想破人QQ空间密码，于是点击了一个美女的空间，提问：“请问男生最喜欢什么花！”我暗...,2012-09-03 00:02:10,0,u0AGHHPXK
3,0,0,北京今天，泛起秋意。,2012-09-03 00:07:36,0,uVGJLC4S5
4,0,2,@_@,2012-09-03 02:09:32,0,uATZPTH53


In [3]:
df2 = df.drop('hashtag_count', axis = 1)
df2 = df2.drop('mention_count', axis = 1)
df2 = df2.drop('url_count', axis = 1)

In [11]:
"""""
drop = []
for index, row in df2.iterrows():   
    try:   #handle exception when text is null
        if '#' in row['text']:
            drop.append(0)
        else: 
            drop.append(1)
    except:
        drop.append(2)
df2['drop'] = drop
df2.head()

,text,time,userid,drop
0,冷言冷语。。。继续。。。这样很好。。。,2012-09-03 00:03:19,uWWBCGJE4,1
1,傻子 过去就过去了 越想只会让自己 越来越TM看不起自己 恨自己不是吗？有什么了不起的 人不...,2012-09-03 00:04:12,u1O52TEBM,1
2,闲来无事，想破人QQ空间密码，于是点击了一个美女的空间，提问：“请问男生最喜欢什么花！”我暗...,2012-09-03 00:02:10,u0AGHHPXK,1
3,北京今天，泛起秋意。,2012-09-03 00:07:36,uVGJLC4S5,1
4,@_@,2012-09-03 02:09:32,uATZPTH53,1


In [12]:
"""""
df2 = df2[df2['drop'] == 0]
df2 = df2.drop('drop', axis = 1)
df2.head()

,text,time,userid
40,@uDGUXVVZY： 【七絕 晚霞】#平水韻#晚霞雲彩滿天紅，曠野無垠陣陣風，似海波濤輕湧...,2012-09-03 00:21:28,uK3RUJX1C
85,#有粉有态度 健康粉女郎#我刚刚在 @ukn： 报名了#万名抹茶粉女郎全国大招募活动#，快来...,2012-09-03 14:40:48,uTZ5HLFF5
110,"#爱情公寓#爱是当你爱上一个人，会舍弃自己的自由，换取他的自由。爱上当你爱上一个人,会改变自...",2012-09-03 00:18:37,uZ32PKRL0
125,#静观NewsDesign# 另外一部分原因，在于设计师对于图片中信息的理解欠火候，野蛮的处...,2012-09-03 00:32:18,u351ZNLSP
126,#静观NewsDesign# 图片和图形的结合，一直是这几年编辑们不敢触碰的领域，其主要障碍...,2012-09-03 00:29:05,u351ZNLSP


In [4]:
print len(df2)
df2.head()

892539


,text,time,userid
0,冷言冷语。。。继续。。。这样很好。。。,2012-09-03 00:03:19,uWWBCGJE4
1,傻子 过去就过去了 越想只会让自己 越来越TM看不起自己 恨自己不是吗？有什么了不起的 人不...,2012-09-03 00:04:12,u1O52TEBM
2,闲来无事，想破人QQ空间密码，于是点击了一个美女的空间，提问：“请问男生最喜欢什么花！”我暗...,2012-09-03 00:02:10,u0AGHHPXK
3,北京今天，泛起秋意。,2012-09-03 00:07:36,uVGJLC4S5
4,@_@,2012-09-03 02:09:32,uATZPTH53


## Extract hashtag

In [7]:
hashtags = []
for index, row in df2.iterrows():
    text = str(row['text'])   
    if text.find('#') < 0:  # no hashtag found
        hashtags.append('')
    else:   # message contains '#'
        hashtags_per = []  
        count = 0
        while count < 10:
            start = text.find('#') 
            if text[start+1:].find('#') < 0:
                break  
            else:
                end = text.find('#', start+1)
                hashtag = text[start+1: end]    #   #idea # is not a hashtag; #idea# is
                if ' ' not in hashtag:
                    hashtags_per.append(hashtag)
                text = text[end+1:]
                count += 1
                if text.find('#')<0:
                    break
        hashtags.append(hashtags_per)

In [39]:
print len(hashtags)
hs = []
count = 0
for h in hashtags:
    if h != '':
        hs.append((h, count))
    count += 1
print len(hs)
print hs[43][0][0], hs[43][1]

892539
23353
李成种生日快乐 2841


In [12]:
df2['hashtag'] = hashtags

In [18]:
df2[40:50]

,text,time,userid,hashtag
40,@uDGUXVVZY： 【七絕 晚霞】#平水韻#晚霞雲彩滿天紅，曠野無垠陣陣風，似海波濤輕湧...,2012-09-03 00:21:28,uK3RUJX1C,[平水韻]
41,@风格豹：【豹起阑栅意】曾问长领似蝤蛴，朦胧细腰隔汤霓。冰笋不调绿云乱，长裙似水自高低。玉柔...,2012-09-03 00:19:49,uK3RUJX1C,
42,Jennifer1962：紫色的云河，飘浮着，梦一样的光绚，翩跹在，我黑色的眼眸里。一道异彩...,2012-09-03 00:09:07,uK3RUJX1C,
43,謝謝鼓勵！感恩！ //@uDGUAHFWB： 过去不重要，重要的是把握现在和未来[鼓掌] /...,2012-09-03 00:27:04,uBIBS02HN,
44,[给力] //@u3FTLGANE： 笑哥威武[帅]//@uBIBS02HN： [给力] /...,2012-09-03 00:14:39,uBIBS02HN,
45,🙏好樣的 //@uW0E3EZEB： 我捐助3个孩子！！！让爱延续……支持公益，传递爱心,2012-09-03 00:14:03,uBIBS02HN,
46,[给力] //@uHJXP0DXY： 笑哥！苍劲有力的嗓音无人能替代[拳头] //@uBIB...,2012-09-03 00:12:03,uBIBS02HN,
47,回复@uRSRGHTLL： 謝謝 感恩 //@uRSRGHTLL： @uBIBS02HN： ...,2012-09-03 00:11:38,uBIBS02HN,
48,會努力的 謝謝 //@uHRVTXGRH： [围观] 希望更多的好作品出来！//@uBIBS...,2012-09-03 00:08:24,uBIBS02HN,
49,謝謝 感動,2012-09-03 00:06:19,uBIBS02HN,


# Chinese text segementation

In [25]:
#remove hashtag from each message
text_rem_hash = []
for index, row in df2.iterrows():
    text = row['text']
    hashtag = row['hashtag']
    for h in hashtag:
        text = text.replace(h, '')
    text_rem_hash.append(text)

In [26]:
print len(text_rem_hash)
print text_rem_hash[125]

892539
## 另外一部分原因，在于设计师对于图片中信息的理解欠火候，野蛮的处理方式对于图片的伤害会极大。


In [28]:
# remove @mention
text_rem_at = []
for text in text_rem_hash:
    text = str(text)
    count = 0
    while count < 20:
        if text.find('@') < 0:
            break
        else:
            start = text.find('@')
            if text.find('：', start+1) < 0: # @ is at the end of the message
                text = text[:start]
                break
            else:
                end = text.find('：', start+1)
                text = text[:start]+text[end+3:]
        count += 1    
    text_rem_at.append(text)

In [30]:
print len(text_rem_hash)
print len(text_rem_at)
print text_rem_hash[2756]
print text_rem_at[2756]

892539
892539
有点英倫feel很喜欢[花心]//@ukn：  ## ## 哇 ~~~~ //@uWWBBUYJV：  我这里还没到[泪]
有点英倫feel很喜欢[花心]//  ## ## 哇 ~~~~ //  我这里还没到[泪]


In [31]:
# remove url
text_rem_url = []
for text in text_rem_at:
    text = str(text)
    count = 0
    while count < 10:
        if text.find('http') < 0:
            break  
        else:
            start = text.find('http')
            if text.find(' ', start+1) < 0:  # url in the end
                text = text[:start]
            else:                            # url in the middle
                end = text.find(' ')  
                text = text[:start]+text[end+1:]
                if text.find('http') < 0:
                    break
            count += 1
    text_rem_url.append(text)

In [34]:
print text_rem_at[2641]
print text_rem_url[2641]

 ## “地址：http://t.cn/zWms4AF
 ## “地址：


In [35]:
# remove punctuation
punc = ['#', '，', '。', '！', '？', '：', '“', '”', '（', '）', '【', '】', '+', '、', '％', '.', ':', '(', ')', '[', ']', '《', '》', '', '；', '～', '＂']
punc = punc+[',', '!', '?', '+', '=', '-', '~', '"', '<', '>', '/', '^', ';', '//']
text_rem_punc = []
for text in text_rem_url:
    text = text.replace(' ', '')
    for c in punc:
        if c in text:
            text = text.replace(c, '')            
    text_rem_punc.append(text)

In [36]:
print text_rem_url[2641]
print text_rem_punc[2641]

 ## “地址：
地址


In [37]:
df2['cleaned_text'] = text_rem_punc
df3 = df2.drop('text', axis = 1)
df3.head()

,time,userid,hashtag,cleaned_text
0,2012-09-03 00:03:19,uWWBCGJE4,,冷言冷语继续这样很好
1,2012-09-03 00:04:12,u1O52TEBM,,傻子过去就过去了越想只会让自己越来越TM看不起自己恨自己不是吗有什么了不起的人不都是在挫折中...
2,2012-09-03 00:02:10,u0AGHHPXK,,闲来无事想破人QQ空间密码于是点击了一个美女的空间提问请问男生最喜欢什么花我暗香肯定是玫瑰因...
3,2012-09-03 00:07:36,uVGJLC4S5,,北京今天泛起秋意
4,2012-09-03 02:09:32,uATZPTH53,,


# Save to file for later indexing

In [41]:
hashtags2 = []
for hashtags_per in hashtags:
    i = ''
    for hashtag in hashtags_per:
        i += hashtag+' '
    hashtags2.append(i)

print len(hashtags2)
print hashtags2[2841]

892539
李成种生日快乐 李成种生日快乐 


In [42]:
df3.drop('hashtag', axis = 1)
df3['hashtag'] = hashtags2
df3.head()

,time,userid,hashtag,cleaned_text
0,2012-09-03 00:03:19,uWWBCGJE4,,冷言冷语继续这样很好
1,2012-09-03 00:04:12,u1O52TEBM,,傻子过去就过去了越想只会让自己越来越TM看不起自己恨自己不是吗有什么了不起的人不都是在挫折中...
2,2012-09-03 00:02:10,u0AGHHPXK,,闲来无事想破人QQ空间密码于是点击了一个美女的空间提问请问男生最喜欢什么花我暗香肯定是玫瑰因...
3,2012-09-03 00:07:36,uVGJLC4S5,,北京今天泛起秋意
4,2012-09-03 02:09:32,uATZPTH53,,


In [ ]:
df3.to_csv('ready_for_index_long')

# Combine overseas users

In [3]:
df = pd.read_csv('ready_for_index_long')
f1 = open('oversea_users.csv')
csv_f1 = csv.reader(f1)

oversea = []
for row in csv_f1:
    oversea.append(row[0])
    
oversea_user_set = set(oversea)
print len(oversea_user_set)

is_oversea = []
for index, row in df.iterrows():
    if row['userid'] in oversea_user_set:
        is_oversea.append(1)
    else:
        is_oversea.append(0)
df['is_oversea'] = is_oversea
print len(df)
df.head()

26010
892539


,Unnamed: 0,time,userid,hashtag,cleaned_text,is_oversea
0,0,2012-09-03 00:03:19,uWWBCGJE4,NaN,冷言冷语继续这样很好,0
1,1,2012-09-03 00:04:12,u1O52TEBM,NaN,傻子过去就过去了越想只会让自己越来越TM看不起自己恨自己不是吗有什么了不起的人不都是在挫折中...,1
2,2,2012-09-03 00:02:10,u0AGHHPXK,NaN,闲来无事想破人QQ空间密码于是点击了一个美女的空间提问请问男生最喜欢什么花我暗香肯定是玫瑰因...,0
3,3,2012-09-03 00:07:36,uVGJLC4S5,NaN,北京今天泛起秋意,0
4,4,2012-09-03 02:09:32,uATZPTH53,NaN,NaN,0


In [4]:
df.to_csv('ready_for_index_long')